# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,33.39,55,65,21.34,TF,1719962499
1,1,edinburgh of the seven seas,-37.0676,-12.3116,58.73,86,79,37.33,SH,1719962499
2,2,malabar,-33.9667,151.2500,55.85,87,20,19.57,AU,1719962499
3,3,isafjordur,66.0755,-23.1240,43.90,79,95,6.93,IS,1719962410
4,4,lebu,-37.6167,-73.6500,48.20,78,92,10.38,CL,1719962499


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha=0.75
)

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 69)]

#Wind speed less than 3.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 3.5)]

#Not a cloud in the sky 
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
161,161,remire-montjoly,4.9167,-52.2667,77.04,100,0,2.30,GF,1719962510
254,254,roma,43.2128,-75.4557,77.72,52,0,3.44,US,1719962516
317,317,coxim,-18.5067,-54.7600,77.56,30,0,2.44,BR,1719962521
330,330,jijel,36.8205,5.7667,69.75,68,0,3.44,DZ,1719962477
352,352,martinsville,36.6915,-79.8725,84.18,34,0,3.44,US,1719962523


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/05/hqdn865j4plg9vq4vjst54t00000gn/T/ipykernel_36457/2638726515.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
161,remire-montjoly,GF,4.9167,-52.2667,100,
254,roma,US,43.2128,-75.4557,52,
317,coxim,BR,-18.5067,-54.7600,30,
330,jijel,DZ,36.8205,5.7667,68,
352,martinsville,US,36.6915,-79.8725,34,
423,san borja,BO,-14.8167,-66.8500,72,
497,ascencion de guarayos,BO,-15.8963,-63.1881,48,
511,lata,PT,40.1629,-8.3327,80,
515,djamaa,DZ,33.5339,5.9931,27,
553,chegdomyn,RU,51.1167,133.0167,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
roma - nearest hotel: Wingate by Wyndham Rome
coxim - nearest hotel: Avendita
jijel - nearest hotel: Hôtel Louiza
martinsville - nearest hotel: Baymont Martinsville
san borja - nearest hotel: No hotel found
ascencion de guarayos - nearest hotel: Hotel Ɨguasú
lata - nearest hotel: Residencial Botânico
djamaa - nearest hotel: Rose d’or
chegdomyn - nearest hotel: Бурея


,City,Country,Lat,Lng,Humidity,Hotel Name
161,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
254,roma,US,43.2128,-75.4557,52,Wingate by Wyndham Rome
317,coxim,BR,-18.5067,-54.7600,30,Avendita
330,jijel,DZ,36.8205,5.7667,68,Hôtel Louiza
352,martinsville,US,36.6915,-79.8725,34,Baymont Martinsville
423,san borja,BO,-14.8167,-66.8500,72,No hotel found
497,ascencion de guarayos,BO,-15.8963,-63.1881,48,Hotel Ɨguasú
511,lata,PT,40.1629,-8.3327,80,Residencial Botânico
515,djamaa,DZ,33.5339,5.9931,27,Rose d’or
553,chegdomyn,RU,51.1167,133.0167,51,Бурея


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)